In [77]:
import os, shutil, re, datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
from scipy.signal import resample

%matplotlib inline

In [78]:
# Change VARS
PERSON = "taylor"
TRYS = 5
SAVED_DATA = "/Users/42robotics/Documents/Processing/OpenBCI_GUI/OpenBCI_GUI/SavedData"
NB_CHANNELS = 4

# Const VARS
DATE = datetime.datetime.today().strftime('%m_%d')
FNAME = "{}{}_{}".format(PERSON,TRYS,DATE)
RESAMPLE = True
ROOT = os.getcwd() + "/openbci_data/raw_data"
AUD_ROOT = os.getcwd() + "/openbci_data/openbci_to_wav/"
OUTPUT = AUD_ROOT + FNAME
IMG_EXT = ".png"
VERBOSITY = 1000

In [79]:
def newest(path):
    files = [f for f in os.listdir(path) if not f.startswith('.')]
    paths = [os.path.join(path, basename) for basename in files]
    return max(paths, key=os.path.getctime)

In [80]:
def channel2wav(ch_data, fname, channel):
    data = np.array(ch_data, dtype='float64')
    print("channel {} max: {}".format(channel, np.max(np.abs(data))))
    data /= np.max(np.abs(data))
    if RESAMPLE == True:
        data = resample(data, len(data) * 40)
    wavfile.write(fname + "/" + channel + '.wav', 201, data)
    print("{} channel wav file conversion done!".format(channel))

In [81]:
# Get latest recorded file from OpenBCI savedData folder

newFile_path = newest(SAVED_DATA)
newFile = open(newFile_path, 'r')
lines = newFile.readlines()[6:-1]
newFile.close()

if FNAME not in os.listdir(AUD_ROOT):
    os.mkdir(AUD_ROOT + FNAME)

fixed = open(AUD_ROOT + FNAME + "/" + FNAME, 'w')
fixed.write("id,ch1,ch2,ch3,ch4,na1,na2,na3,timestamp\n")
for line in lines:
    fixed.write(line)
fixed.close()

df = pd.read_csv(AUD_ROOT + FNAME + "/" + FNAME, ",")

# Convert txtfile into wavfiles from ch1 to NB_CHANNELS

for ch in range(1,NB_CHANNELS+1):
    ch_df = df['ch{}'.format(ch)].tolist()
    channel2wav(ch_df, OUTPUT, "0{}".format(ch))

# cleanup
if os.path.exists(AUD_ROOT + FNAME + "/" + FNAME):
    os.remove(AUD_ROOT + FNAME + "/" + FNAME)

channel 01 max: 21.05
01 channel wav file conversion done!
channel 02 max: 49.69
02 channel wav file conversion done!
channel 03 max: 19.1
03 channel wav file conversion done!
channel 04 max: 31.29
04 channel wav file conversion done!
